Visualizing our Simulations
========================

Each time we run our Maestro study, we add 10 new runs of the ball bouncing simulation to our datastore. Now let's take a look at those results...

In [ ]:
from numbers import Number
from collections import defaultdict

import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

import sina.datastores.sql as sina_sql
import sina.utils
from sina.datastore import create_datastore
from sina.visualization import Visualizer

# For notebook interactive widgets
# %matplotlib notebook

# For lab turn off interactive widgets
%matplotlib inline

# use this instead of notebook if using vscode, unless you install the matplotlib extension
#%matplotlib widget

# Initialization
database = 'output.sqlite'
target_type = "csv_rec"
datastore = create_datastore(database)
recs = datastore.records
vis = Visualizer(datastore)
print("Sina is ready!")

Sanity Check: are our runs randomized?
========

We want to make sure our starting positions are properly randomized. There should be a variety of values in this graph.

In [ ]:
_= vis.create_histogram("x_vel_initial", interactive=True)

Simple hypothesis testing
===============

Could there be a relationship between having a high initial velocity and a high number of bounces?

In [ ]:
_ = vis.create_scatter_plot("y_vel_initial", "num_bounces", interactive=True, matplotlib_options={"color": "orange", "alpha": 0.5, "s": 80})

Actually watching a ball bounce
===========================

Since we're collecting position at each step, we can easily visualize how the ball bounching actually works...we'll select the ball with the high number of bounces for interest.

In [ ]:
id_pool = list(recs.find_with_max("num_bounces", 1, ids_only=True))
_ = vis.create_line_plot("x_pos", "y_pos", id_pool=id_pool)

Scaling Up
==========

Each run within a set shares a few starting conditions, but differs in several others. Let's see a whole set together in 3D space to compare their performances. First, let's see what groups we have available.

In [ ]:
groups = list(set(x["group_id"]["value"] for x in recs.get_data(["group_id"]).values()))
print("Found the following groups: {}".format(groups))

Pick whichever you like, set `GROUP_OF_INTEREST` equal to it, and watch the "fireworks"!

This 3D plot isn't (yet) a formal part of Sina, but it's simple enough.

In [ ]:
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt

GROUP_OF_INTEREST = list(groups)[0]
id_pool = list(recs.find_with_data(group_id=GROUP_OF_INTEREST))

ball_set_fig = plt.figure()
ball_set_ax = plt.axes(projection='3d')

for rec_id in id_pool:
    curveset = recs.get(rec_id).get_curve_set("physics_cycle_series")
    # Some slight weirdness due to 3D axis labeling...simulation thinks of y as up-down
    ball_set_ax.plot3D(curveset.get_dependent("x_pos")["value"],
                       curveset.get_dependent("z_pos")["value"],
                       curveset.get_dependent("y_pos")["value"],
                       label=rec_id)
    ball_set_ax.set_xlabel('X')
    ball_set_ax.set_zlabel('Y')
    ball_set_ax.set_ylabel('Z')
_ = ball_set_ax.legend()

Close Curve Analysis with PyDV
=========================

While Sina allows you to survey large numbers of runs at once, it doesn't include many operations on individual curves. PyDV works directly with Sina, though, opening the door to some extremely powerful manipulations.

## PyDV Plotting

In [ ]:
# Note: pydv's big! Importing it may take a moment.
from pydv import pydvpy

# PyDV operates on files. We'll dump one of Sina's records to a file
PYDV_DEMO_FILENAME = "pydv_demo_sina.json"
rec = list(recs.get_all())[0]
rec.to_file(PYDV_DEMO_FILENAME)

# For more information on PyDV's Python API visit https://pydv.readthedocs.io/en/latest/pydv.html
curves = pydvpy.readsina(PYDV_DEMO_FILENAME)

# Plotting with matplotlib
plt.style.use('ggplot')
for cur in curves:
    # print(cur.__dict__)
    plt.plot(cur.x, cur.y, label=cur.name)
    
plt.title("My Plot")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend(loc="upper right")
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10.1, 11.3)
fig.savefig('myPlot1.png', dpi=100)


In [ ]:
# Or you can use PyDV's built in plotting capabilities which are built on top of matplotlib
plot2 = pydvpy.create_plot(curves, fname='myPlot2', ftype='png', fwidth=10.1, fheight=11.3, title='My Plot', xlabel='X', ylabel='Y', legend=True, stylename='ggplot')

## PyDV Maths

In [ ]:
# Take the cosine of y values of a Curve or list of Curves.
pydvpy.cos(curves)
plot3 = pydvpy.create_plot(curves, fname='myPlot2 cos', ftype='png', fwidth=10.1, fheight=11.3, title='My Plot', xlabel='X', ylabel='Y', legend=True, stylename='ggplot')

In [ ]:
# Take the absolute value of the y values of the Curve or list of curves.
pydvpy.abs(curves)
plot4 = pydvpy.create_plot(curves, fname='myPlot2 cos abs', ftype='png', fwidth=10.1, fheight=11.3, title='My Plot', xlabel='X', ylabel='Y', legend=True, stylename='ggplot')

In [ ]:
# Add one or more curves.
c = pydvpy.add(curves)
plot5 = pydvpy.create_plot(c, fname='myPlot2 cos abs add', ftype='png', fwidth=10.1, fheight=11.3, title='My Plot', xlabel='X', ylabel='Y', legend=True, stylename='ggplot')